In [1]:
import gradio as gr
import pickle

/home/hafizh/miniconda3/envs/MainCuda/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
with open(f"./model/model.pkl", "rb") as f:
    model = pickle.load(f)

In [ ]:
def get_yes_no_question(yes_no):
    yes_no_mapping = {
        "No" : 0,
        "Yes" : 1
    }
    return yes_no_mapping.get(yes_no, -1)

def get_gender(gender):
    gender_mapping = {
        "Female" : 0,
        "Male" : 1
    }
    return gender_mapping.get(gender, -1)

def get_mother_occupation(mother):
    mother_occupation_mapping = {
        "Student" : 0,
        "Specialists in Intellectual and Scientific Activities" : 2,
        "Intermediate Level Technicians and Professions" : 3,
        "Administrative Staff" : 4,
        "Personal Services, Security and Safety Workers and Sellers" : 5,
        "Skilled Workers in Industry, Construction and Craftsmen" : 7,
        "Unskilled Workers" : 9, 
        "Other" : 999
    }
    return mother_occupation_mapping.get(mother, -1)

def predict_status(_sem_enrolled, _scholarship_holder, _sem_approved, _sem_credited, 
                _tuition_fees, _sem_evaluations, _gender, _admission_grade, 
                _age, _sem_without_evaluations, _mothers_occupation, _gdp):
    scholarship_holder = get_yes_no_question(_scholarship_holder)
    tuition_fees = get_yes_no_question(_tuition_fees)
    gender = get_gender(_gender)
    mothers_occupation = get_mother_occupation(_mothers_occupation)
    data = [[_sem_enrolled, scholarship_holder, _sem_approved, _sem_credited,
             tuition_fees, _sem_evaluations, gender, _admission_grade,
             _age, _sem_without_evaluations, mothers_occupation, _gdp]]
    prediction = model.predict(data)#[0]
    # prediction_proba = model.predict_proba(data)[0][prediction]
    return prediction

array(['Curricular_units_2nd_sem_enrolled', 'Scholarship_holder',
       'Curricular_units_2nd_sem_approved',
       'Curricular_units_2nd_sem_credited', 'Tuition_fees_up_to_date',
       'Curricular_units_2nd_sem_evaluations', 'Gender',
       'Admission_grade', 'Age_at_enrollment',
       'Curricular_units_2nd_sem_without_evaluations',
       'Mothers_occupation', 'GDP'], dtype='<U44')

In [ ]:
with gr.Blocks(title="Student Performance Prediction") as demo:
    gr.Markdown("""
                # 🎒 Student Performance Prediction
                # Dicoding - Solving Educational Institution Problem
                ## Made by : Muhammad Hafizh Dzaki
                ## Gihub Repo : [Here](https://github.com/hfzdzakii/Dicoding-SolvingEducationIntsituteProblem)
                """)
    with gr.Row():
        with gr.Column():
            gr.Markdown("### Input Variables")
            sem_enrolled = gr.Number(label="Sum of 2nd Semester Curricular Units Enrolled:",
                                     value=0, minimum=0, maximum=24)
            sem_approved = gr.Number(label="Sum of 2nd Semester Curricular Units Approved:",
                                     value=0, minimum=0, maximum=24)
            sem_credited = gr.Number(label="Sum of 2nd Semester Curricular Units Credited:",
                                     value=0, minimum=0, maximum=24)
            sem_evaluations = gr.Number(label="Sum of 2nd Semester Curricular Units Evalutions:",
                                        value=0, minimum=0)
            sem_without_evaluations = gr.Number(label="Sum of 2nd Semester Curricular Units Without Evalutions:",
                                                value=0, minimum=0)
            scholarship_holder = gr.Radio(label="Scholarship Holder?",
                                          choices=["No", "Yes"], value="No")
            tuition_fees = gr.Radio(label="Tuition Fees Payed?",
                                    choices=["No", "Yes"], value="No")
            gender = gr.Radio(label="Gender:", 
                              choices=["Male", "Female"], value="Male")
            admission_grade = gr.Number(label="Admission Grade:",
                                        value=0, minimum=0, maximum=200)
            age = gr.Number(label="Admission Grade:",
                            value=0, minimum=18)
            mothers_occupation = gr.Dropdown(label="Mother Occupation:",
                                             choices=["Student", "Specialists in Intellectual and Scientific Activities", "Intermediate Level Technicians and Professions",
                                                      "Administrative Staff", "Personal Services, Security and Safety Workers and Sellers",
                                                      "Skilled Workers in Industry, Construction and Craftsmen", "Unskilled Workers", "Other"],
                                             value="Student")
            gdp = gr.Number(label="Gross Domestic Product (GDP):",
                            value=0)
            
        with gr.Column():
            gr.Markdown("""### Example Data
                        Choose one from list below to fill input immediately!
                        """)
            
            gr.Markdown("### Predict and Result")
            predict_button = gr.Button(label="Predict", variant="primary")
            prediction = gr.Textbox(label="Prediction", interactive=False)
            
    predict_button.click(
        fn=predict_status,
        inputs=[sem_enrolled, scholarship_holder,
                sem_approved,
                sem_credited, tuition_fees,
                sem_evaluations, gender,
                admission_grade, age,
                sem_without_evaluations,
                mothers_occupation, gdp],
        outputs=prediction,
    )
            